In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG19
import threading
from flask_cors import CORS

2025-02-26 10:59:17.431619: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
app = Flask(__name__)
CORS(app) 

In [3]:
model = load_model("skin_disease_model.h5", compile=False)

In [4]:
# Define categories
categories = [
    "Acne",
    "Eczema",
    "Hair Loss, Alopecia",
    "Nail Fungus",
    "Scabies",
]

In [5]:

vgg_model = VGG19(weights="imagenet", include_top=False, input_shape=(180, 180, 3))

In [6]:

for layer in vgg_model.layers:
    layer.trainable = False

In [7]:
def preprocess_image(image):
    """Preprocess image for prediction."""
    img = cv2.imdecode(np.frombuffer(image.read(), np.uint8), cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (180, 180)) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)

    # Extract features using VGG19
    img_features = vgg_model.predict(img)
    img_features = img_features.reshape(1, -1)  # Flatten

    return img_features

In [8]:
@app.route("/predict", methods=["POST"])
def predict():
    """API endpoint to predict skin disease from an uploaded image."""
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    img_features = preprocess_image(file)

    # Predict disease
    prediction = model.predict(img_features)[0]
    predicted_class = np.argmax(prediction)
    confidence = round(np.max(prediction) * 100, 2)

    return jsonify({"disease": categories[predicted_class], "confidence": confidence})

In [9]:
def run_flask():
    app.run(host="0.0.0.0", port=5001, debug=True, use_reloader=False)

In [10]:
if __name__ == "__main__":
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.68.75:5001
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


127.0.0.1 - - [26/Feb/2025 10:59:50] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


...

127.0.0.1 - - [26/Feb/2025 11:10:03] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


127.0.0.1 - - [26/Feb/2025 11:10:26] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


127.0.0.1 - - [26/Feb/2025 11:10:39] "POST /predict HTTP/1.1" 200 -
